In [1]:
import numpy as np
import pandas as pd

In [2]:
#Load dataset
df = pd.read_csv("train.csv")

In [3]:
prediction = ["Lemon?"]
predictor = list(df.drop(prediction, axis=1))

# Feature Extraction

In [4]:
#Remove uncessary Features
predictor.remove("WheelTypeID")
predictor.remove("RefId")
predictor.remove("Model")
predictor.remove("Trim")
predictor.remove("SubModel")
predictor.remove("Color")
predictor.remove("PRIMEUNIT")
predictor.remove("AUCGUART")
predictor.remove("VNZIP1")
predictor.remove("Date Purchased")
predictor.remove("BYRNO")
predictor

['Auction',
 'Vehicle Year',
 'Vehicle Age',
 'Make',
 'Transmission',
 'WheelType',
 'VehOdo',
 'Nationality',
 'Size',
 'TopThreeAmericanName',
 'MMRAcquisitionAuctionAveragePrice',
 'MMRAcquisitionAuctionCleanPrice',
 'MMRAcquisitionRetailAveragePrice',
 'MMRAcquisitonRetailCleanPrice',
 'MMRCurrentAuctionAveragePrice',
 'MMRCurrentAuctionCleanPrice',
 'MMRCurrentRetailAveragePrice',
 'MMRCurrentRetailCleanPrice',
 'VNST',
 'VehBCost',
 'IsOnlineSale',
 'WarrantyCost']

In [5]:
#fill in missing value


#Fill in missing string values
X = df[predictor]
values = {'Transmission': 'AUTO', 'WheelType': 'Alloy', 'Nationality': 'AMERICAN', 'Size': 'MEDIUM', 'TopThreeAmericanName':'GM'}
X = X.fillna(value=values)

#Transmission seems to have 'Manual and 'MANUAL', replace 'Manual' with all caps
X = X.replace('Manual', 'MANUAL')
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43789 entries, 0 to 43788
Data columns (total 22 columns):
Auction                              43789 non-null object
Vehicle Year                         43789 non-null int64
Vehicle Age                          43789 non-null int64
Make                                 43789 non-null object
Transmission                         43789 non-null object
WheelType                            43789 non-null object
VehOdo                               43789 non-null int64
Nationality                          43789 non-null object
Size                                 43789 non-null object
TopThreeAmericanName                 43789 non-null object
MMRAcquisitionAuctionAveragePrice    43775 non-null float64
MMRAcquisitionAuctionCleanPrice      43775 non-null float64
MMRAcquisitionRetailAveragePrice     43775 non-null float64
MMRAcquisitonRetailCleanPrice        43775 non-null float64
MMRCurrentAuctionAveragePrice        43598 non-null float64
MMRC

In [6]:
#fill missing value with mean
X = X.fillna(X.mean())
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43789 entries, 0 to 43788
Data columns (total 22 columns):
Auction                              43789 non-null object
Vehicle Year                         43789 non-null int64
Vehicle Age                          43789 non-null int64
Make                                 43789 non-null object
Transmission                         43789 non-null object
WheelType                            43789 non-null object
VehOdo                               43789 non-null int64
Nationality                          43789 non-null object
Size                                 43789 non-null object
TopThreeAmericanName                 43789 non-null object
MMRAcquisitionAuctionAveragePrice    43789 non-null float64
MMRAcquisitionAuctionCleanPrice      43789 non-null float64
MMRAcquisitionRetailAveragePrice     43789 non-null float64
MMRAcquisitonRetailCleanPrice        43789 non-null float64
MMRCurrentAuctionAveragePrice        43789 non-null float64
MMRC

In [7]:
#One hot encode
string_variables =['Auction', 'Make', 'Transmission', 'WheelType', 'Nationality', 'Size', 'TopThreeAmericanName','VNST']
X = pd.get_dummies(X, prefix=string_variables)

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43789 entries, 0 to 43788
Columns: 111 entries, Vehicle Year to VNST_WV
dtypes: float64(9), int64(5), uint8(97)
memory usage: 8.7 MB


In [9]:
#Split validation set and training set
from sklearn.model_selection import train_test_split

y =df[prediction]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=153,
                                                    stratify=y)

In [10]:
#Scale data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Modelling with Keras Neural Network

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers

model = Sequential()

model.add(Dense(111, activation='relu', input_dim=111))

model.add(Dense(74, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(50, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(33,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(22,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(14, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(9, activation='relu'))

model.add(Dense(1, activation='relu'))

Using TensorFlow backend.


In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3, batch_size=1, verbose=1)

Epoch 1/3
35031/35031 [==============================] - 39s 1ms/step - loss: 1.8331 - acc: 0.8654
Epoch 2/3
35031/35031 [==============================] - 42s 1ms/step - loss: 1.9117 - acc: 0.8768
Epoch 3/3
35031/35031 [==============================] - 39s 1ms/step - loss: 1.9751 - acc: 0.8770: 0s - loss: 1.9704 - acc:


In [13]:
score = model.evaluate(X_test, y_test,verbose=1)

print(score)

8758/8758 [==============================] - 0s 30us/step
[1.9820950891597904, 0.87702671845608804]


# Predict

In [14]:
predict = pd.read_csv("predictthis.csv")

In [15]:
X_pred = predict[predictor]